# Exercise 01

## Credit conditions

Making of 2 credit programs.

## Study materials

https://github.com/rouckas/NEVF141/tree/main/2025
Will be updated

## Floating point arithmetic
print float 0.1 with 19 decimal places precision

In [1]:
import sys
print(sys.version)

3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]


In [2]:
import numpy as np

In [3]:
"%.19f"%0.1

'0.1000000000000000056'

In [4]:
f"{0.1:.19f}"

'0.1000000000000000056'

In [5]:
"{:.19f}".format(0.1)

'0.1000000000000000056'

calculate $1+10^{-15}$ a $1+10^{-16}$

In [6]:
1+1e-15

1.000000000000001

In [7]:
1+1e-16

1.0

What is the relative accuracy of a float64 with 53-bit mantissa?

In [8]:
1/2**53

1.1102230246251565e-16

That's how many decimal places?

In [9]:
-np.log10(1/2**(53+1))

16.255619765854984

Calculate $1.001-1$

In [10]:
1.001-1

0.0009999999999998899

Order of operations matters - calculate $10^{-15}+1-1$ and $1-1+10^{-15}$

In [11]:
1e-15+1-1

1.1102230246251565e-15

In [12]:
1-1+1e-15

1e-15

Error amplification: Calculate the terms  of the following sequence for $j=1\ldots10$:

$$x_0 = 1$$
$$x_j = (1.01 - x_{j-1})\cdot 100$$

and compare with hand calculation...

In [13]:
x = 1
# task 1
for j in range(1, 11):
    x = (1.01-x)*100
    print(j, x)

1 1.0000000000000009
2 0.9999999999999121
3 1.0000000000087939
4 0.9999999991206154
5 1.0000000879384574
6 0.9999912061542604
7 1.0008793845739605
8 0.912061542603948
9 9.7938457396052
10 -878.38457396052


Can we improve accuracy with np.float128?

In [14]:
import numpy as np

In [15]:
x = 1
y = np.float128(101)/100
for j in range(1, 11):
    x = (y-x)*100
    print(j, x)

1 0.99999999999999999913
2 1.0000000000000000859
3 0.99999999999999141225
4 1.000000000000858774
5 0.9999999999141226002
6 1.0000000085877399801
7 0.99999914122600199167
8 1.000085877399800832
9 0.99141226001991679655
10 1.8587739980083203438


Yes, but not by much, as float64 is actually just 80 bit on most machines

In [16]:
np.float128(1.01), np.float128(101)/100

(1.0100000000000000089, 1.01)

We may use arbitrary precision floats from the mpmath library

In [17]:
from mpmath import mp
mp.dps = 32 # pracujeme na 32 desetinnych mist
x = 1
y = mp.mpf("1.01")
for j in range(1, 11):
    x = (y-x)*100
    print(j, x)

1 0.99999999999999999999999999999998
2 1.0000000000000000000000000000018
3 0.99999999999999999999999999981694
4 1.0000000000000000000000000183059
5 0.99999999999999999999999816941312
6 1.0000000000000000000001830586878
7 0.99999999999999999998169413122167
8 1.0000000000000000018305868778334
9 0.99999999999999981694131221665877
10 1.000000000000018305868778334123


In [18]:
# use interval arithmetics to obtain rigorous error bounds
from mpmath import mp, iv
iv.dps = 16 # 16 decimal digits to compare with float64 calculation
x = 1
y = iv.mpf("1.01")
for j in range(1, 11):
    x = (y-x)*100
    print(j, x, 1 in x) # on every step, check that the true solution is within the calculated interval

1 [0.999999999999998112621, 1.00000000000000088818] True
2 [0.999999999999909294779, 1.00000000000018962609] True
3 [0.99999999998103550336, 1.00000000000907141029] True
4 [0.99999999909285708366, 1.00000000189645055215] True
5 [0.999999810354942897384, 1.00000009071429252216] True
6 [0.999990928570745896131, 1.00001896450571114983] True
7 [0.998103549428883130012, 1.00090714292541127506] True
8 [0.909285707458870606779, 1.18964505711168788693] True
9 [-17.9645057111687909135, 10.0714292541129402103] True
10 [-906.142925411294044125, 1897.45057111687913221] True


## Series summation
Calculate 

$$y = \sum_{i=0}^{1000} \frac{1}{10^{-6} + i}$$

 - in order
 - in reverse order
 - using the builtin `sum` and `math.fsum`

and compare the results

In [19]:
x = np.arange(1001)
y = 1/(1e-6 + x)

In order

In [20]:
s = 0
for i in y: s += i
s

1000007.4854692194

Reverse order

In [21]:
s = 0
for i in y[::-1]: s += i
s

1000007.4854692166

Builtin sum just adds everything in order (until Python 3.12, when they switched to Neumaier sum)

In [22]:
sum(y)
# note: builtin sum behavior has changed as of Python 3.12 
# https://docs.python.org/3/library/functions.html#sum

1000007.4854692194

math.fsum is "almost" guaranteed to be exact, see https://docs.python.org/3/library/math.html#math.fsum , https://code.activestate.com/recipes/393090-binary-floating-point-summation-accurate-to-full-p/

In [23]:
import math
math.fsum(y)

1000007.4854692166

As a compromise between speed and accuracy, numpy.sum may use partial pairwise summation https://numpy.org/doc/2.3/reference/generated/numpy.sum.html

In [24]:
np.sum(y)

1000007.4854692165

Which answer is correct? Check with Sympy

In [25]:
import sympy as sp
si = sp.Symbol("si")
series = sp.Sum(1/(sp.S(10)**(-6) + si), (si, 0, 1000))
series

Sum(1/(si + 1/1000000), (si, 0, 1000))

In [26]:
series.evalf(30)

1000007.48546921661698028641805

## Number comparison

Number comparison: check equality of:

$1+ 10^{-16}$ and $1$;

$(1.01-1)\cdot100$ and $1$

In [27]:
1+1e-16 == 1

True

In [28]:
1 == (1.01 - 1)*100

False

Due to roundoff error, equality comparisons are complicated. Better to check equality with limited relative and absolute error, e.g., `isclose`

In [29]:
def isclose(a, b, rel_tol=1e-9, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [30]:
isclose(1, (1.01 - 1)*100)

True

Or better use the library functions `np.isclose` or `math.isclose`, that guarrantee correct behavior with inf or NaN...

In [31]:
np.isclose(1, (1.01 - 1)*100)

True

## "Special" numbers: zero, infinity, NaN

Python is dynamically typed - name is not bound to a data type, like in C/C++. The variable type can be queried with `type` function.

In [32]:
x = 0
type(x)

int

In [33]:
x = 0.0
type(x)

float

A note about number types in Python: calculate `1/0`, `1/0.0` and `1/np.sin(0)`. Why do the results differ?

In [34]:
1/0

ZeroDivisionError: division by zero

In [35]:
1/0.0

ZeroDivisionError: float division by zero

In [36]:
1/np.sin(0)

/tmp/ipykernel_900128/1905035236.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  1/np.sin(0)


inf

The problem lies in different floating point number types. Numpy float allows division by zero - it will just emit warning and correctly return inf. This numpy behavior can be modified.

64-bit numpy zero float is created as `np.float64(0)`. This data type can also represent Nan (not a number, undefined expression), +inf, -inf (infinities), +0, -0 (signed zero).

In [37]:
-np.float64(0), np.nan, -np.inf, np.inf

(-0.0, nan, -inf, inf)

Some sample NaN calculations:

In [38]:
1/np.float64(0), 1/(-np.float64(0))

/tmp/ipykernel_900128/3417162462.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  1/np.float64(0), 1/(-np.float64(0))


(inf, -inf)

In [39]:
0/np.float64(0)

/tmp/ipykernel_900128/608735182.py:1: RuntimeWarning: invalid value encountered in scalar divide
  0/np.float64(0)


nan

All comparisons with NaN are false.

In [40]:
np.nan > 3, np.nan <= 3, np.nan == np.nan

(False, False, False)

In [41]:
np.inf == np.inf

True

In [42]:
-np.float64(0) == np.float64(0)

True

In [43]:
np.arctan(np.inf)/np.pi

0.5

In [44]:
np.inf > -np.nan

False